In [6]:
import pandas as pd
import numpy as np
items = pd.read_csv("../reporting/powerbi-share/R_outputs/items_by_order.csv")
items = items[1:10]

In [7]:
items.head()
items.shape

(9, 2)

In [8]:
print "max num of items", np.max(items.V1.apply(lambda s: len(s.split(","))))
print "index of order with max num of items", np.argmax(items.V1.apply(lambda s: len(s.split(","))))
max_num_items = np.max(items.V1.apply(lambda s: len(s.split(","))))

max num of items 6
index of order with max num of items 4


In [9]:
import gensim
model = gensim.models.Word2Vec(items.values, size=100, window=max_num_items, min_count=2, workers=4)

RuntimeError: you must first build vocabulary before training the model

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(model.wv.syn0)

In [ ]:
def get_batch(vocab, model, n_batches=3):
    output = list()
    for i in range(0, n_batches):
        rand_int = np.random.randint(len(vocab), size=1)[0]
        suggestions = model.most_similar(positive=[vocab[rand_int]], topn=5)
        suggest = list()
        for i in suggestions:
            suggest.append(i[0])
        output += suggest
        output.append(vocab[rand_int])
    return output
def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    """From Tensorflow's tutorial."""
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(21, 21))  #in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i,:]
        plt.scatter(x, y)
        plt.annotate(label,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
from matplotlib import pyplot as plt
embeds = []
labels = []
for item in get_batch(vocab, model, n_batches=5):
    embeds.append(pca.transform(model[item])[0])
    labels.append(products.loc[int(item)]['product_name'])
embeds = np.array(embeds)
plot_with_labels(embeds, labels)